<a href="https://colab.research.google.com/github/garfield-gray/Optimization/blob/main/NonConvex/Cholesky_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
import numpy as np

#Implementations

In [175]:

def choleskyFactorization(A):
    """
    Recursive Cholesky decomposition of a symmetric positive definite matrix.

    Parameters:
    A (numpy.ndarray): Input matrix.

    Returns:
    numpy.ndarray: Lower triangular matrix (L) such that A = LL^T.
    """
    n = A.shape[0]
    L = np.zeros_like(A, dtype=np.float64)

    if n == 1:
        # l11 in the book
        L[0, 0] = np.sqrt(A[0, 0])
    else:
        # l11 in the book
        L[0, 0] = np.sqrt(A[0, 0])
        # l21 in the book
        L[1:n, 0] = A[1:n, 0] / L[0, 0]
        # l22 in the book
        L[1:n, 1:n] = choleskyFactorization(A[1:n, 1:n] - np.outer(L[1:n, 0], L[1:n, 0]))

    return L


In [176]:
import numpy as np

def forward_substitution(L, b):
    """
    Solve Ly = b for y using forward substitution.

    Parameters:
    L (numpy.ndarray): Lower triangular matrix.
    b (numpy.ndarray): Right-hand side vector.

    Returns:
    numpy.ndarray: Solution vector y.
    """
    n = L.shape[0]
    y = np.zeros_like(b, dtype=np.float64)

    for i in range(n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]

    return y

def back_substitution(L, y):
    """
    Solve L^Tx = y for x using back substitution.

    Parameters:
    L (numpy.ndarray): Lower triangular matrix.
    y (numpy.ndarray): Right-hand side vector.

    Returns:
    numpy.ndarray: Solution vector x.
    """
    n = L.shape[0]
    x = np.zeros_like(y, dtype=np.float64)

    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(L[i,i+1:], x[i+1:])) / L[i,i]

    return x

def solveEquation(A, b):
    """
    Solve the equation Ax = b using the Cholesky decomposition.

    Parameters:
    A (numpy.ndarray): Left-hand side matrix
    b (numpy.ndarray): Right-hand side vector.

    Returns:
    numpy.ndarray: Solution vector x.
    """
    # Step 0: calculating L (numpy.ndarray): Lower triangular matrix (Cholesky decomposition of A).
    L = choleskyFactorization(A)

    # Step 1: Solve Ly = b for y using forward substitution
    y = forward_substitution(L, b)

    # Step 2: Solve L^Tx = y for x using back substitution
    x = back_substitution(L.T, y)

    return x


In [177]:
def MSE(H) -> int:
  return np.mean((H)**2)

#Tests

##CholeskeyTest

In [178]:
M = np.array([[9, 4, 3],
              [4, 17, 21],
              [9, 21, 100]])
np.linalg.cholesky(M)

array([[3.        , 0.        , 0.        ],
       [1.33333333, 3.90156664, 0.        ],
       [3.        , 4.35722405, 8.48614156]])

In [179]:
choleskyFactorization(M)

array([[3.        , 0.        , 0.        ],
       [1.33333333, 3.90156664, 0.        ],
       [3.        , 4.35722405, 8.48614156]])

In [180]:
mse = MSE(np.linalg.cholesky(M) - choleskyFactorization(M))
print(mse)

8.765121169122353e-32


##EquationTest

In [181]:
b = ([3, 3.0030, 4.0010])
A = np.array([[1, 1, 1],
              [1, 1.001, 1.001],
              [1, 1.001, 2]])

In [182]:
np.linalg.solve(A, b)

array([-4.44533299e-13,  2.00100100e+00,  9.98998999e-01])

In [183]:
solveEquation(A, b)

array([-4.44089210e-13,  2.00100100e+00,  9.98998999e-01])

In [184]:
mse = MSE(np.linalg.solve(A, b) - solveEquation(A, b))
print(mse)

6.573840876841765e-32
